<a href="https://colab.research.google.com/github/creativesar/Hangman-project/blob/main/hangman_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import string
import os
import time

def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

def get_word(difficulty="medium"):
    # Expanded word list with categories
    word_categories = {
        'Programming': ['python', 'javascript', 'algorithm', 'variable', 'function', 'database', 'developer',
                       'compiler', 'debugging', 'framework', 'interface', 'library', 'method', 'object'],
        'Animals': ['elephant', 'giraffe', 'penguin', 'dolphin', 'kangaroo', 'leopard', 'octopus', 'rhinoceros',
                   'squirrel', 'tiger', 'zebra', 'koala', 'flamingo', 'hedgehog'],
        'Countries': ['australia', 'brazil', 'canada', 'denmark', 'egypt', 'france', 'germany', 'india',
                     'japan', 'mexico', 'netherlands', 'portugal', 'sweden', 'thailand'],
        'Movies': ['avatar', 'inception', 'titanic', 'jaws', 'gladiator', 'frozen', 'matrix', 'alien'],
        'Sports': ['football', 'basketball', 'tennis', 'swimming', 'volleyball', 'cricket', 'hockey', 'rugby']
    }

    # Select a random category
    category = random.choice(list(word_categories.keys()))
    words = word_categories[category]

    # Filter words based on difficulty
    if difficulty == "easy":
        filtered_words = [w for w in words if len(w) <= 6]
    elif difficulty == "medium":
        filtered_words = [w for w in words if 6 < len(w) <= 8]
    else:  # hard
        filtered_words = [w for w in words if len(w) > 8]

    # If no words match the difficulty, use all words from the category
    if not filtered_words:
        filtered_words = words

    word = random.choice(filtered_words)
    return word.upper(), category

def display_hangman(lives):
    stages = [
        # Final state: head, torso, both arms, both legs
        '''
           --------
           |      |
           |      O
           |     \\|/
           |      |
           |     / \\
           -
        ''',
        # 1 life left: head, torso, both arms, one leg
        '''
           --------
           |      |
           |      O
           |     \\|/
           |      |
           |     /
           -
        ''',
        # 2 lives left: head, torso, both arms
        '''
           --------
           |      |
           |      O
           |     \\|/
           |      |
           |
           -
        ''',
        # 3 lives left: head, torso, one arm
        '''
           --------
           |      |
           |      O
           |     \\|
           |      |
           |
           -
        ''',
        # 4 lives left: head and torso
        '''
           --------
           |      |
           |      O
           |      |
           |      |
           |
           -
        ''',
        # 5 lives left: head
        '''
           --------
           |      |
           |      O
           |
           |
           |
           -
        ''',
        # 6 lives left: initial state
        '''
           --------
           |      |
           |
           |
           |
           |
           -
        '''
    ]
    return stages[lives]

def get_hint(word, used_letters):
    # Find a letter that hasn't been guessed yet
    for letter in word:
        if letter not in used_letters:
            return letter
    return None

def calculate_score(word, lives_remaining, difficulty):
    # Base score depends on word length and lives remaining
    base_score = len(word) * 10 + lives_remaining * 20

    # Difficulty multiplier
    if difficulty == "easy":
        difficulty_multiplier = 1
    elif difficulty == "medium":
        difficulty_multiplier = 1.5
    else:  # hard
        difficulty_multiplier = 2

    final_score = int(base_score * difficulty_multiplier)
    return final_score

def display_available_letters(alphabet, used_letters):
    available = ""
    for letter in sorted(alphabet):
        if letter in used_letters:
            available += letter + " "
        else:
            available += "_ "
    return available

def hangman(difficulty="medium"):
    word, category = get_word(difficulty)
    word_letters = set(word)  # letters in the word
    alphabet = set(string.ascii_uppercase)
    used_letters = set()  # what the user has guessed

    lives = 6  # Number of lives
    hints_remaining = 2  # Number of hints available

    # Getting user input
    while len(word_letters) > 0 and lives > 0:
        clear_screen()

        # Display hangman
        print(display_hangman(lives))

        # Display game info
        print(f'Category: {category}')
        print(f'Difficulty: {difficulty.capitalize()}')
        print(f'You have {lives} lives left')
        print(f'Hints remaining: {hints_remaining}')

        # Display available letters
        print(f'\nAvailable letters: {display_available_letters(alphabet, used_letters)}')

        # What current word is (ie W - R D)
        word_list = [letter if letter in used_letters else '_' for letter in word]
        print(f'\nCurrent word: {" ".join(word_list)}')

        # Get user input
        print("\nOptions:")
        print("- Enter a letter to guess")
        if hints_remaining > 0:
            print("- Type 'hint' to get a hint")
        print("- Type 'quit' to exit the game")

        user_input = input('\nYour choice: ').upper()

        # Handle special commands
        if user_input == 'QUIT':
            print("\nGame aborted. Returning to main menu...")
            time.sleep(1.5)
            return None

        elif user_input == 'HINT' and hints_remaining > 0:
            hints_remaining -= 1
            hint_letter = get_hint(word, used_letters)
            if hint_letter:
                print(f"\nHint: The word contains the letter '{hint_letter}'")
                used_letters.add(hint_letter)
                if hint_letter in word_letters:
                    word_letters.remove(hint_letter)
            else:
                print("\nNo more hints available!")
                hints_remaining += 1  # Give the hint back if no letter to hint
            input('Press Enter to continue...')
            continue

        # Normal letter guess
        if len(user_input) != 1 or user_input not in alphabet:
            print('\nPlease enter a single valid letter.')
            input('Press Enter to continue...')
            continue

        if user_input in alphabet - used_letters:
            used_letters.add(user_input)
            if user_input in word_letters:
                word_letters.remove(user_input)
                print(f'\nGood job! {user_input} is in the word.')
                time.sleep(0.5)
            else:
                lives = lives - 1  # takes away a life if wrong
                print(f'\nSorry, {user_input} is not in the word.')
                input('Press Enter to continue...')
        elif user_input in used_letters:
            print('\nYou have already used that letter. Try another one.')
            input('Press Enter to continue...')

    # Final state display
    clear_screen()
    print(display_hangman(lives))

    # Calculate score if player won
    if lives > 0:
        score = calculate_score(word, lives, difficulty)
        print(f'Congratulations! You guessed the word {word}!!')
        print(f'You had {lives} lives remaining.')
        print(f'Your score: {score}')
        return score
    else:
        print(f'You lost! The word was {word}')
        input('\nPress Enter to continue...')
        return None

def main():
    while True:
        clear_screen()
        print("=" * 50)
        print("WELCOME TO HANGMAN!")
        print("=" * 50)
        print("\nMAIN MENU:")
        print("1. Start Game (Easy)")
        print("2. Start Game (Medium)")
        print("3. Start Game (Hard)")
        print("4. Exit")

        choice = input("\nEnter your choice (1-4): ")

        if choice == '1':
            hangman("easy")
            input("\nPress Enter to continue...")
        elif choice == '2':
            hangman("medium")
            input("\nPress Enter to continue...")
        elif choice == '3':
            hangman("hard")
            input("\nPress Enter to continue...")
        elif choice == '4':
            print("\nThanks for playing! Goodbye!")
            break
        else:
            print("\nInvalid choice. Please try again.")
            input("Press Enter to continue...")

if __name__ == "__main__":
    main()

WELCOME TO HANGMAN!

MAIN MENU:
1. Start Game (Easy)
2. Start Game (Medium)
3. Start Game (Hard)
4. Exit
